In [ ]:
%load_ext autoreload
%autoreload 2

import sys
# Add to Python path Mitsuba python binding
sys.path.insert(0, '/home/mcrespo/mitsuba3/build/python')
# Add to Python path Custom packages
sys.path.append('/home/mcrespo/PyPackages')

# Initialize Mitsuba and variant
import mitsuba as mi
mi.set_variant('cuda_rgb')

# Extra imports
import drjit as dr
import numpy as np
import pprint
import matplotlib.pyplot as plt

# Import Mitsuba transient package - should be after importing Mitsuba and Dr.JIT
import MiTransient as mitr

# Load the scene

In [ ]:
kernel_scene = mi.load_file('../scenes/cornellBox/cbox_mirror.xml')
# kernel_scene = mi.load_file('../scenes/cbox/cbox-media.xml')
# kernel_scene = mi.load_file('../scenes/cbox/transient.xml')
transient_integrator = kernel_scene.integrator()

# Prepare the transient storage and other transient mechanisms

In [ ]:
transient_integrator.prepare_transient(kernel_scene, 0)

# Render the scene

In [ ]:
img = transient_integrator.render(kernel_scene, spp=64)

> Plot the resulting non-transient image

In [ ]:
plt.figure(figsize=(9,9))
plt.imshow(img ** (1.0 / 2.2))
plt.show()

# Develop the transient data `[pixelX, pixelY, time channels]`

In [ ]:
data_transient = transient_integrator.transientBlock.develop(raw=False, gamma=False)
print(data_transient.shape)

# Save resulting transient data into frames

In [ ]:
import sys
import pyexr

import os.path
from os import path
if not path.exists("transientFrames"):
    os.makedirs("transientFrames")

print("Saving transient frames")
transientTensor = np.array(data_transient, dtype=float)
for id in range(transientTensor.shape[2]):
    transientData = np.array(np.power(transientTensor[:,:,id,:], 1/2.2) * 255, dtype=float)
    transientData = np.rot90(transientData, k=3)
    transientData = np.flip(transientData, axis=1)

    pyexr.write('transientFrames/{0:03d}.exr'.format(id), transientData)
    sys.stdout.write("\r\tSaved %d frame" % (id))
    sys.stdout.flush()

# Test streak images

In [ ]:
PATH = '/home/mcrespo/PyPackages/MiTransient/scenes/cbox/transient_scene'

import os

list_files = os.listdir(PATH)
list_files = sorted(list_files, key=lambda name: int(name[6:-4]))

import pyexr
img = pyexr.read(os.path.join(PATH, list_files[0]))
print(img.shape)


In [ ]:
streak_data = np.zeros([512, 512, 400, 4])
index = 0
for file in list_files:
    img = pyexr.read(os.path.join(PATH, file))
    streak_data[index, :, :, :] = img
    index += 1

In [ ]:
plt.figure()
plt.imshow(streak_data[:,:,0,0:3])
plt.show()

In [ ]:
import sys
import pyexr
print("Saving transient frames")
transientTensor = np.array(streak_data, dtype=float)
for id in range(transientTensor.shape[2]):
    transientData = np.array(np.power(transientTensor[:,:,id,0:3], 1/2.2) * 255, dtype=float)
    transientData = np.nan_to_num(transientData)
    # transientData = np.rot90(transientData, k=3)
    # transientData = np.flip(transientData, axis=1)

    pyexr.write('transientFrames/{0:03d}.exr'.format(id), transientData)
    sys.stdout.write("\r\tSaved %d frame" % (id))
    sys.stdout.flush()